In [1]:
import itertools
import numpy as np
import plotly.express as plx
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from scipy.spatial import Voronoi, voronoi_plot_2d

from function_splitting import crop_axis, form_x_y_polygons
from functions_voronoi import voronoi_finite_polygons_2d, voronoi_polygons

from Bezier_functions import skew_polygon, skew_graph, evaluate_bezier

import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(suppress=True)
%load_ext autoreload
%autoreload 2

In [2]:
INPUT_WIDTH = 100
INPUT_HEIGHT = 100

in_points = np.array([[x, y] for x in range(INPUT_WIDTH) for y in range(INPUT_HEIGHT)])
in_points.shape

(10000, 2)

In [28]:
N_points = 10

kmeans = KMeans(n_clusters=N_points, random_state=0)
clusters = kmeans.fit_predict(in_points)
new_points = np.array([np.mean(in_points[clusters == curr], axis=0) for curr in set(clusters)])
new_points += np.array([[x_r, y_r] for x_r,y_r in zip(np.random.randint(N_points, size=N_points), 
                                                      np.random.random(N_points)+np.random.randint(N_points, size=N_points))])
new_points.shape

(10, 2)

In [29]:
polygons = voronoi_polygons(new_points)
cropped_polygons = crop_axis(polygons, range_x=(0, INPUT_WIDTH), range_y=(0, INPUT_HEIGHT))
x_polygs, y_polygs = form_x_y_polygons(cropped_polygons)

fig = go.Figure(go.Scatter(x=x_polygs, y=y_polygs, mode='lines+markers'))
fig.show()

In [30]:
polygons = skew_graph(cropped_polygons, 0, INPUT_WIDTH, 0, INPUT_HEIGHT)
x_polygs, y_polygs = form_x_y_polygons(polygons)

x0_polygs, y0_polygs = form_x_y_polygons(cropped_polygons)


fig = go.Figure([
    go.Scatter(x=x_polygs, y=y_polygs, mode='lines'),
    go.Scatter(x=x0_polygs, y=y0_polygs, mode='markers')
])
fig.show()